# MONTANA DATA REVIEW

#### Items needing fact checking are denoted in the draft by an "FC" comment. Check them here, denoting each with the sentence/fact you are checking in the draft. Update the related FC comment in the draft when you confirm or deny the finding.

##### TO-DO's per Matt's review
* TODO get all stops
* TODO filter to consent searches
* TODO group by race, get consent search rate (searches/stops)
* TODO derive arrest rate for consent-searched NA, white drivers (arrests/searches)
* TODO filter by geography (flathead, or the district it belongs to)
* TODO rerun consent search rate for flathead
* TODO etc

### Setup

In [76]:
# Libraries

library(tidyverse)
library(lubridate)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



In [24]:
# Parameters

### File paths
mt_path <- "data/hp256wp2687_mt_statewide_2019_08_13.rds"

In [77]:
# Load Data

data <-
    mt_path %>%
    read_rds()

In [88]:
data <-
    data %>%
    mutate(
        year = year(date)
    )

In [89]:
head(data)

raw_row_number,date,time,location,lat,lng,county_name,subject_age,subject_race,subject_sex,⋯,reason_for_stop,vehicle_make,vehicle_model,vehicle_type,vehicle_registration_state,vehicle_year,raw_Race,raw_Ethnicity,raw_SearchType,year
<chr>,<date>,<drtn>,<chr>,<dbl>,<dbl>,<chr>,<int>,<fct>,<fct>,⋯,<chr>,<chr>,<chr>,<chr>,<fct>,<int>,<chr>,<chr>,<chr>,<dbl>
1,2009-01-01,02:10:53,US 89 N MM10 (SB),47.57274,-111.8029,Cascade County,16,white,female,⋯,--- - HIT AND RUN,FORD,EXPLORER,SPORT UTILITY,MT,1994,W,N,NO SEARCH REQUESTED,2009
2,2009-01-02,11:34:19,HWY 93 SO AND ANNS LANE S/B,46.76123,-114.0811,Missoula County,19,white,male,⋯,EXPIRED TAG ( - MONTHS OR LESS ),GMC,TK,TRUCK,MT,1996,W,N,NO SEARCH REQUESTED,2009
3,2009-01-03,11:36:42,P007 HWY 93 MM 77 N/B,46.69468,-114.0735,Missoula County,17,white,male,⋯,SPEED,GMC,YUKON,SPORT UTILITY,MT,1999,W,N,NO SEARCH REQUESTED,2009
4,2009-01-04,10:33:11,P007 HWY 93 MM 81 S/B,46.72739,-114.0790,Missoula County,17,NA,female,⋯,SPEED,HOND,CR-V,SPORT UTILITY,MT,2002,W,NA,NA,2009
5,2009-01-04,10:46:43,P007 HWY 93 MM 81 N/B,46.72952,-114.0791,Missoula County,31,NA,male,⋯,SPEED,TOYT,TERCEL,SEDAN,MT,1992,I,NA,NA,2009
6,2009-01-04,14:41:57,P007 HWY 93 MM 67 S/B,46.53228,-114.1107,Ravalli County,20,NA,male,⋯,OTHER TAG/REGISTRATION VIOLATIONS,JEEP,CHE,SPORT UTILITY,MT,1998,W,NA,NA,2009


In [34]:
data %>% colnames()

[1] "raw_row_number"             "date"                      
 [3] "time"                       "location"                  
 [5] "lat"                        "lng"                       
 [7] "county_name"                "subject_age"               
 [9] "subject_race"               "subject_sex"               
[11] "department_name"            "type"                      
[13] "violation"                  "arrest_made"               
[15] "citation_issued"            "warning_issued"            
[17] "outcome"                    "frisk_performed"           
[19] "search_conducted"           "search_basis"              
[21] "reason_for_stop"            "vehicle_make"              
[23] "vehicle_model"              "vehicle_type"              
[25] "vehicle_registration_state" "vehicle_year"              
[27] "raw_Race"                   "raw_Ethnicity"             
[29] "raw_SearchType"

In [90]:
data %>% count(search_basis)

Warning message:
“Factor `search_basis` contains implicit NA, consider using `forcats::fct_explicit_na`”

search_basis,n
<fct>,<int>
plain view,137
consent,2240
probable cause,43
other,593
NA,822105


### Review

#### 1) Native American & White Consent Search Rate for 2010-2016

SENTENCE: "Between 2010 and 2016, officers searched 10 of every 1,000 Native American drivers they stopped — more than four times the rate at which they searched white drivers."

CHECKING:
* NA consent search rate
* White consent search rate

In [91]:
one_data <- 
    data %>%
    group_by(raw_Race) %>%
    summarise(
        total_stops = n(),
        consent_searches = sum(search_basis == "consent", na.rm = TRUE),
        consent_search_rate = (consent_searches/total_stops) * 100
    )

one_data

raw_Race,total_stops,consent_searches,consent_search_rate
<chr>,<int>,<int>,<dbl>
A,6700,33,0.49253731
B,8806,93,1.05609811
I,38795,388,1.00012888
U,2634,2,0.07593014
W,768175,1724,0.22442803
NA,8,0,0.00000000


In [92]:
one_na_csearch_rate <-
    one_data %>%
    filter(raw_Race == "I") %>%
    pull(consent_search_rate) %>%
    round(digits = 4)

one_w_csearch_rate <-
    one_data %>%
    filter(raw_Race == "W") %>%
    pull(consent_search_rate) %>%
    round(digits = 4)

paste0("Native American Consent Search Rate: ", one_na_csearch_rate, "%")
paste0("White Consent Search Rate: ", one_w_csearch_rate, "%")

paste0(
    "Native American drivers are subjected to consent searches ",
    round(one_na_csearch_rate/one_w_csearch_rate, digits = 1), 
    "x more often than White drivers."
)

[1] "Native American Consent Search Rate: 1.0001%"

[1] "White Consent Search Rate: 0.2244%"

[1] "Native American drivers are subjected to consent searches 4.5x more often than White drivers."

#### 2) Change in Native American & White Consent Search Rate from 2010 to 2016

SENTENCE: "While search rates for Native American and white drivers were closer together in 2010, by 2016, the two search rates had diverged sharply: The search rate of Native American drivers across the state was six times the search rate of white drivers."

CHECKING:

* NA and White consent search rate in 2010
* NA and White consent search rate in 2016

In [117]:
two_data <-
    data %>%
    group_by(year, raw_Race) %>%
    filter(raw_Race %in% c("W", "I"), !is.na(year)) %>%
    summarise(
        total_stops = n(),
        consent_searches = sum(search_basis == "consent", na.rm = TRUE),
        consent_search_rate = (consent_searches/total_stops) * 100
    ) %>%
    select(-total_stops, -consent_searches) %>%
    spread(key = raw_Race, value = consent_search_rate) %>%
    mutate(na_to_white_search_proportion = round(I/W, digits = 1))

two_data

year,I,W,na_to_white_search_proportion
<dbl>,<dbl>,<dbl>,<dbl>
2009,0.2190581,0.2206736,1.0
2010,0.2104525,0.1743717,1.2
2011,0.4107178,0.1486680,2.8
2012,0.3376365,0.1470148,2.3
2013,0.6026600,0.1867257,3.2
2014,1.8712575,0.3032028,6.2
2015,1.5897048,0.3214373,4.9
2016,1.9351167,0.3196767,6.1


In [128]:
for (y in two_data$year) {
    
    two_na_csearch_rate <- two_data %>% filter(year == y) %>% pull(na_to_white_search_proportion)
    
    paste0("In ", y, " Native Americans were searched at ", two_na_csearch_rate, "x the rate that white drivers were searched at.") %>% print()
    
}

[1] "In 2009 Native Americans were searched at 1x the rate that white drivers were searched at."
[1] "In 2010 Native Americans were searched at 1.2x the rate that white drivers were searched at."
[1] "In 2011 Native Americans were searched at 2.8x the rate that white drivers were searched at."
[1] "In 2012 Native Americans were searched at 2.3x the rate that white drivers were searched at."
[1] "In 2013 Native Americans were searched at 3.2x the rate that white drivers were searched at."
[1] "In 2014 Native Americans were searched at 6.2x the rate that white drivers were searched at."
[1] "In 2015 Native Americans were searched at 4.9x the rate that white drivers were searched at."
[1] "In 2016 Native Americans were searched at 6.1x the rate that white drivers were searched at."
